In [3]:
#!/usr/bin/env python

"""Sample setup file"""

from __future__ import absolute_import
from __future__ import print_function

import os
import shutil
import subprocess
import numpy as np
# import mesh
import matplotlib.pyplot as plt 
import scipy.stats as stats
import decimal
import pixiedust
#import MESH model for branching channel with rec.&trap. x-section with lateral inflow as msh
import mc_sc_stime as mc
from time import process_time 
#%load_ext line_profiler
#@profile


Pixiedust database opened successfully


In [4]:
# %%pixie_debugger 

#*** WARNING: All Python variables corresponding to Fortran module must be in lowercase.
# fortran source code that is used for f2py transformation includes do k=1, ntim loop so 
# that multiple timestep computation is activated inside of the f2py module.

ntim=2;       #the number of time steps necessary for variables passed to mc module to compute successfully
nlinks=2;     #the number of links needed to define varialbe qd. ** nlinks is not used in fortran source code.
                                    
ncomp0=3; mc.var.ncomp0=ncomp0  #the number of segments of a reach upstream of the current reach
ncomp=3; mc.var.ncomp=ncomp  #the number of segments of the current reach 
mxseg=max(ncomp0,ncomp)
mc.var.uslinkid=1
mc.var.linkid=2

#MC model outputs
mc.var.qd=np.zeros((ntim,mxseg,nlinks))  #will store MC output qdc (flow downstream current timestep) 
mc.var.vela=np.zeros((ntim,ncomp)) 
mc.var.deptha=np.zeros((ntim,ncomp))


#lateral flow
mc.var.qlat=np.zeros((ncomp))

dt=60.0;     mc.var.dt= dt
dx=20.0;     mc.var.dx= dx
bw=50;       mc.var.bw= bw
tw= 0.01*bw; mc.var.tw= tw
twcc=tw;     mc.var.twcc=twcc
n=0.03;      mc.var.n=n
ncc=n;       mc.var.ncc=ncc
cs=1.0e6;    mc.var.cs=cs
so=0.002;    mc.var.so=so



#run M-C model
nts=10  #the number of timestep in simulation
wnlinks=20 #the number of all links in simulation
wmxseg=5   #max number of segments among all links

#variable storing all outputs in time
wqd= np.zeros((nts,wmxseg,wnlinks))   
wvela= np.zeros((nts,wmxseg,wnlinks)) 
wdeptha= np.zeros((nts,wmxseg,wnlinks))


          

for k in range (0,nts):        
    #input lateral flow for current reach; input flow to upstream reach of current reach
    for i in range(0,ncomp):
        mc.var.qlat[i]= (k+1)*2.0
        mc.var.qd[1,i,0]= (k+1)*10.0
    
    mc.mc.main()
        
    for i in range(0,ncomp):
        #current link(=reach)
        #qd[k,i,j]: k=0/1: previous/current timestep; i: node ID; j=0/1: upstream/current reach
        mc.var.qd[0,i,1]= mc.var.qd[1,i,1]
        mc.var.vela[0,i]= mc.var.vela[1,i]
        mc.var.deptha[0,i]= mc.var.deptha[1,i]
        #upstream link(=reach)        
        mc.var.qd[0,i,0]=  mc.var.qd[1,i,0] 
    
    #output keeping
    j=1 #temporarily assigned link ID for the current reach
    for i in range(0,ncomp):
        wqd[k,i,j]= mc.var.qd[1,i,1]
        wvela[k,i,j]= mc.var.vela[1,i]
        wdeptha[k,i,j]= mc.var.deptha[1,i]
        
#test output    
j=1
for k in range (0,nts):
    for i in range(0,ncomp):
        print(wqd[k,i,j])
        
for k in range (0,nts):
    for i in range(0,ncomp):
        print(wvela[k,i,j])

for k in range (0,nts):
    for i in range(0,ncomp):
        print(wdeptha[k,i,j])



0.7494052648544312
0.7494052648544312
0.7494052648544312
9.422990798950195
3.3677287101745605
3.3677287101745605
22.68459701538086
13.01193904876709
8.167729377746582
34.936920166015625
27.150066375732422
18.443098068237305
46.98738479614258
41.37955093383789
33.40867614746094
58.997474670410156
55.4658203125
49.3853759765625
70.99949645996094
69.49114227294922
65.4497299194336
82.9999008178711
83.49783325195312
81.48286437988281
94.99998474121094
97.49949645996094
97.4948501586914
107.0
111.49989318847656
113.49856567382812
0.09288035333156586
0.09288035333156586
0.09288035333156586
0.14602933824062347
0.14602933824062347
0.14602933824062347
0.20427046716213226
0.20427046716213226
0.20427046716213226
1.0854473114013672
0.9801607728004456
0.8409706354141235
1.2230900526046753
1.1640342473983765
1.0660821199417114
1.3376725912094116
1.3053598403930664
1.2475899457931519
1.4382942914962769
1.426185965538025
1.3927719593048096
1.5287978649139404
1.5322802066802979
1.5176705121994019
1.611